# Solving an operation with StorageSystemSimulations

> **Set up**
>
> To run this notebook, first install the Julia kernel for Jupyter Notebooks using [IJulia](https://julialang.github.io/IJulia.jl/stable/manual/installation/), then [create an environment](https://pkgdocs.julialang.org/v1/environments/) for this tutorial with the packages listed with `using <PackageName>` further down.
>
> This tutorial has demonstrated compatibility with these package versions. If you run into any errors, first check your package versions for consistency using `Pkg.status()`.
>
 > ```
 > Status `~/work/StorageSystemsSimulations.jl/StorageSystemsSimulations.jl/docs/Project.toml`
 >   [336ed68f] CSV v0.10.16
 >   [a93c6f00] DataFrames v1.8.1
 >   [864edb3b] DataStructures v0.19.3
 >   [e30172f5] Documenter v1.17.0
 >   [d12716ef] DocumenterInterLinks v1.1.0
 >   [35a29f4d] DocumenterTools v0.1.21
 >   [87dc4568] HiGHS v1.21.1
 >   [2cd47ed4] InfrastructureSystems v3.3.1
 >   [23fbe1c1] Latexify v0.16.10
 >   [98b081ad] Literate v2.21.0
 >   [e690365d] PowerSimulations v0.33.0
 >   [f00506e0] PowerSystemCaseBuilder v2.2.0
 >   [bcd98974] PowerSystems v5.5.0
 >   [08abe8d2] PrettyTables v3.2.3
 >   [e2f1a126] StorageSystemsSimulations v0.15.0 `~/work/StorageSystemsSimulations.jl/StorageSystemsSimulations.jl`
 > 
 > ```


**Originally Contributed by**: Jose Daniel Lara

## Introduction

## Load Packages

In [ ]:
using PowerSystems
using PowerSimulations
using StorageSystemsSimulations
using PowerSystemCaseBuilder
using HiGHS ## solver

## Data

> *Note*
>
>
> `PowerSystemCaseBuilder.jl` is a helper library that makes it easier to reproduce examples in the documentation and tutorials. Normally you would pass your local files to create the system data instead of calling the function `build_system`.
> For more details visit [`PowerSystemCaseBuilder.jl`](https://nrel-sienna.github.io/PowerSystemCaseBuilder.jl/stable)

In [ ]:
c_sys5_bat = build_system(
    PSITestSystems,
    "c_sys5_bat_ems";
    add_single_time_series = true,
    add_reserves = true,
)
orcd = get_component(ReserveDemandCurve, c_sys5_bat, "ORDC1")
set_available!(orcd, false)

batt = get_component(EnergyReservoirStorage, c_sys5_bat, "Bat2")

operation_cost = get_operation_cost(batt)

template_uc = ProblemTemplate(PTDFPowerModel)
set_device_model!(template_uc, ThermalStandard, ThermalStandardUnitCommitment)
set_device_model!(template_uc, RenewableDispatch, RenewableFullDispatch)
set_device_model!(template_uc, PowerLoad, StaticPowerLoad)
set_device_model!(template_uc, Line, StaticBranch)

storage_model = DeviceModel(
    EnergyReservoirStorage,
    StorageDispatchWithReserves;
    attributes = Dict(
        "reservation" => true,
        "energy_target" => false,
        "cycling_limits" => false,
        "regularization" => true,
    ),
)
set_device_model!(template_uc, storage_model)

set_service_model!(template_uc, ServiceModel(VariableReserve{ReserveUp}, RangeReserve))
set_service_model!(template_uc, ServiceModel(VariableReserve{ReserveDown}, RangeReserve))